In [2]:
# === 0) Mount Google Drive ===
from google.colab import drive
try:
    drive.flush_and_unmount()
except Exception:
    pass
drive.mount('/content/drive', force_remount=True)

# === 1) Set working directory ===
import os, math
os.chdir('/content/drive/MyDrive/digphil')
print("✅ Working in:", os.getcwd())

# === 2) Imports ===
from PIL import Image, ImageOps
from pathlib import Path

# === 3) Paths & settings ===
wordcloud_dir = Path('/content/drive/MyDrive/digphil/wordclouds')
output_png   = Path('/content/drive/MyDrive/digphil/wordclouds_grid_square.png')
output_pdf   = Path('/content/drive/MyDrive/digphil/wordclouds_grid_square.pdf')

# target cell size (all clouds will be fit into this while keeping aspect ratio)
TARGET_W, TARGET_H = 420, 240
PADDING = 6  # minimal space between tiles

# === 4) Load available wordclouds in chapter order (1..19) ===
images = []
order  = []
for i in range(1, 20):
    p = wordcloud_dir / f'chapter_{i}.png'
    if p.exists():
        # Load and convert RGBA to RGB (white bg) if needed
        img = Image.open(p)
        if img.mode in ('RGBA', 'LA'):
            bg = Image.new('RGB', img.size, 'white')
            bg.paste(img, mask=img.split()[-1])
            img = bg
        else:
            img = img.convert('RGB')
        images.append(img)
        order.append(i)
    else:
        print(f"⚠️ Missing: {p.name}")

if not images:
    raise SystemExit("No wordcloud images found. Expected chapter_1.png, chapter_2.png, ... in /wordclouds")

num = len(images)

# === 5) Resize uniformly (preserve aspect ratio, then pad to exact cell) ===
resized = []
for im in images:
    im = ImageOps.contain(im, (TARGET_W, TARGET_H), method=Image.Resampling.LANCZOS)
    # pad to exact TARGET_W x TARGET_H
    pad_w = TARGET_W - im.width
    pad_h = TARGET_H - im.height
    left = pad_w // 2
    top  = pad_h // 2
    im = ImageOps.expand(im, border=(left, top, pad_w - left, pad_h - top), fill='white')
    resized.append(im)

# === 6) Compute a square-ish grid automatically ===
cols = math.ceil(math.sqrt(num))
rows = math.ceil(num / cols)

cell_w, cell_h = TARGET_W, TARGET_H
grid_w = cols * cell_w + (cols - 1) * PADDING
grid_h = rows * cell_h + (rows - 1) * PADDING

# === 7) Compose the mosaic ===
canvas = Image.new('RGB', (grid_w, grid_h), 'white')

for idx, im in enumerate(resized):
    r = idx // cols
    c = idx % cols
    x = c * (cell_w + PADDING)
    y = r * (cell_h + PADDING)
    canvas.paste(im, (x, y))

# === 8) Save outputs (PNG + PDF) ===
canvas.save(output_png, dpi=(300, 300))
canvas.save(output_pdf, "PDF", resolution=300)

print(f"✅ Saved mosaic:\n - {output_png}\n - {output_pdf}")
print(f"🧩 Grid: {rows} rows × {cols} cols | cell {TARGET_W}×{TARGET_H} | padding {PADDING}px")


Mounted at /content/drive
✅ Working in: /content/drive/MyDrive/digphil
✅ Saved mosaic:
 - /content/drive/MyDrive/digphil/wordclouds_grid_square.png
 - /content/drive/MyDrive/digphil/wordclouds_grid_square.pdf
🧩 Grid: 4 rows × 5 cols | cell 420×240 | padding 6px
